# Create Zarr of 9 years of three hourly CSSP China data [1851-1859]

In [1]:
import iris
import os
import sys
import logging
import xarray as xr
import numpy as np

import crd_utils as crd
import umdates_utils as um

from datetime import datetime, timedelta

## Create a list of all the files we want to process

In [2]:
# three hourly data filenames
filepath = '/data/cssp-china/mini-dataset-24-01-19/20CR/3hrly'
files = sorted(os.listdir(filepath))
print(len(files))

5850


In [3]:
# generate all possible filenames for the time period
runid = 'apepd'
startd = datetime(1851, 1, 1)   # 00Z on Jan 01 1851
endd = datetime(1860, 1, 1)     # 00Z on Jan 01 1860
freq = 'pc'

decade_filenames = um.UMFileList(runid, startd, endd, freq)
print(len(decade_filenames))

3288


In [4]:
# find the overlap of actual filenames with all possible filenames in that decade
filenames = list(set(files).intersection(set(decade_filenames)))
filenames.sort()
print(len(filenames))

329


In [5]:
filepaths = [os.path.join(filepath, filename) for filename in filenames]
print(len(filepaths))

329


## Set all the Cube, Dataset and Zarr variables we need to process the cubes

In [6]:
%%time
cubelist = iris.load(filepaths[0:1])
cubelist[0]

CPU times: user 992 ms, sys: 122 ms, total: 1.11 s
Wall time: 1.26 s


<iris 'Cube' of air_pressure_at_sea_level / (Pa) (time: 33; grid_latitude: 219; grid_longitude: 286)>

In [7]:
print(cubelist)

0: air_pressure_at_sea_level / (Pa)    (time: 33; grid_latitude: 219; grid_longitude: 286)
1: air_temperature / (K)               (time: 33; grid_latitude: 219; grid_longitude: 286)
2: cloud_area_fraction / (1)           (time: 32; grid_latitude: 219; grid_longitude: 286)
3: relative_humidity / (%)             (time: 33; grid_latitude: 219; grid_longitude: 286)
4: specific_humidity / (1)             (time: 33; grid_latitude: 219; grid_longitude: 286)
5: surface_air_pressure / (Pa)         (time: 33; grid_latitude: 219; grid_longitude: 286)
6: surface_downwelling_longwave_flux_in_air / (W m-2) (time: 32; grid_latitude: 219; grid_longitude: 286)
7: surface_downwelling_shortwave_flux_in_air / (W m-2) (time: 32; grid_latitude: 219; grid_longitude: 286)


In [8]:
CUBENAMES = [cube.name() for cube in cubelist]
UNIQUE_COORDS = crd.unique_coords_list(cubelist)
COORD_NAME_MAPPING = crd.get_new_coord_names(UNIQUE_COORDS)

In [9]:
print([coord.name() for coord in UNIQUE_COORDS])

['time', 'grid_latitude', 'grid_longitude', 'forecast_reference_time', 'forecast_period', 'height', 'time', 'forecast_period']


In [10]:
COORD_NAME_MAPPING[1]

('time_1', 'forecast_period_1')

In [19]:
ZARR = '/data/cssp-china/zarr_3hourly_1851-1859'
CHUNKS = {'time': 200, 'grid_latitude': 219, 'grid_longitude': 286}

## Initialise logging

In [12]:
LOGFILE = '/data/cssp-china/zarr_append_3hourly.log'
logging.basicConfig(filename=LOGFILE,
                    level=logging.DEBUG,
                    format='%(asctime)s %(message)s',
                    datefmt='%d/%m/%Y %H:%M:%S')

In [13]:
if not os.path.isfile(LOGFILE):
    os.mknod(LOGFILE)

if os.stat(LOGFILE).st_size == 0:
    logging.info('Initiate log')
    print(f'Log initiated at {LOGFILE}')

Log initiated at /data/cssp-china/zarr_append_3hourly.log


## Create a Zarr with 4 cubes, which definitely amounts to 273 time steps (~34 days)

In [15]:
%%time
cubelist0 = iris.load(filepaths[0:4])
cubelist0[1]

CPU times: user 8.18 s, sys: 744 ms, total: 8.93 s
Wall time: 10.4 s


<iris 'Cube' of air_temperature / (K) (time: 273; grid_latitude: 219; grid_longitude: 286)>

In [16]:
%%time
crd.rename_cubes(cubelist0, CUBENAMES, COORD_NAME_MAPPING, dryrun=False)

air_pressure_at_sea_level
  x time
  x grid_latitude
  x grid_longitude
  x forecast_reference_time
  x forecast_period
air_temperature
  x time
  x grid_latitude
  x grid_longitude
  x forecast_reference_time
  x height
  x forecast_period
cloud_area_fraction
  x time
  x grid_latitude
  x grid_longitude
  x forecast_reference_time
  x forecast_period
relative_humidity
  x time
  x grid_latitude
  x grid_longitude
  x forecast_reference_time
  x height
  x forecast_period
specific_humidity
  x time
  x grid_latitude
  x grid_longitude
  x forecast_reference_time
  x height
  x forecast_period
surface_air_pressure
  x time
  x grid_latitude
  x grid_longitude
  x forecast_reference_time
  x forecast_period
surface_downwelling_longwave_flux_in_air
  x time
  x grid_latitude
  x grid_longitude
  x forecast_reference_time
  x forecast_period
surface_downwelling_shortwave_flux_in_air
  x time
  x grid_latitude
  x grid_longitude
  x forecast_reference_time
  x forecast_period
CPU times: us

In [17]:
print(cubelist0)

0: air_pressure_at_sea_level / (Pa)    (time: 273; grid_latitude: 219; grid_longitude: 286)
1: air_temperature / (K)               (time: 273; grid_latitude: 219; grid_longitude: 286)
2: cloud_area_fraction / (1)           (time: 272; grid_latitude: 219; grid_longitude: 286)
3: relative_humidity / (%)             (time: 273; grid_latitude: 219; grid_longitude: 286)
4: specific_humidity / (1)             (time: 273; grid_latitude: 219; grid_longitude: 286)
5: surface_air_pressure / (Pa)         (time: 273; grid_latitude: 219; grid_longitude: 286)
6: surface_downwelling_longwave_flux_in_air / (W m-2) (time: 272; grid_latitude: 219; grid_longitude: 286)
7: surface_downwelling_shortwave_flux_in_air / (W m-2) (time: 272; grid_latitude: 219; grid_longitude: 286)


In [20]:
%%time
dalist0 = crd.cubelist_to_dalist(cubelist0)
ds0 = xr.merge(dalist0)

CPU times: user 222 ms, sys: 0 ns, total: 222 ms
Wall time: 234 ms


In [21]:
ds0

<xarray.Dataset>
Dimensions:                                    (grid_latitude: 219, grid_longitude: 286, time: 545)
Coordinates:
  * time                                       (time) datetime64[ns] 1851-01-01 ... 1851-02-04
  * grid_latitude                              (grid_latitude) float32 22.88 ... -25.08
  * grid_longitude                             (grid_longitude) float32 323.48 ... 386.18002
    forecast_reference_time                    datetime64[ns] 1849-12-01
    forecast_period                            (time) timedelta64[ns] 396 days 00:00:00 ... 430 days 00:00:00
    height                                     float64 1.5
Data variables:
    air_pressure_at_sea_level                  (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(2, 219, 286), meta=np.ndarray>
    air_temperature                            (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(2, 219, 286), meta=np.ndarray>
    cloud_area_fraction                        (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(3, 219, 286), meta=np.ndarray>
    relative_humidity                          (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(2, 219, 286), meta=np.ndarray>
    specific_humidity                          (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(2, 219, 286), meta=np.ndarray>
    surface_air_pressure                       (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(2, 219, 286), meta=np.ndarray>
    surface_downwelling_longwave_flux_in_air   (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(3, 219, 286), meta=np.ndarray>
    surface_downwelling_shortwave_flux_in_air  (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(3, 219, 286), meta=np.ndarray>

# BREAK: Time has interleaved (unknown reasons)
### Will bughunt and find a way around this

-----------

In [ ]:
%%time
logging.info(f'Creating {ZARR}')
crd.ds_to_zarr(ds0, ZARR, chunks=CHUNKS)

In [ ]:
dsz0 = xr.open_zarr(ZARR)
dsz0

In [ ]:
# Size of Dataset in GB
dsz0.nbytes / 1e9

In [ ]:
logging.info(f'  Successfull creation of {ZARR}')
logging.info(f'  Processed filenames 0:{filenames[0]} - 3:{filenames[3]}')
logging.info(f'  Chunking {CHUNKS}')

## Now loop through the remaining cubes and append to the Zarr we created

In [ ]:
def logprint(message):
    logging.info(message)
    print(message)

In [ ]:
%%time
# Loop through in bunches of 2 cubes, so as to not use to much memory at one time
start = 3
step = 2
# stop = len(filepaths)
stop = start+(2*step)

logprint(f'Appending data to {ZARR}')

for i in range(start, stop, step):
    fnames = filenames[i:i+step]
    logprint(f'  Processing filenames {i}:{fnames[0]} - {min([i+step, stop])}:{fnames[-1]}')
    try:
        cubelist = iris.load(filepaths[i:i+step])
        logprint(f'  | Loaded files successfully')
        
        crd.rename_cubes(cubelist, CUBENAMES, COORD_NAME_MAPPING, dryrun=False, verbose=False)
        logprint(f'  | Renamed files successfully')
        
        dalist = crd.cubelist_to_dalist(cubelist)
        ds = xr.merge(dalist)
        logprint(f'  | Created dataset successfully')
        
        crd.ds_to_zarr(ds, ZARR, chunks=CHUNKS)
        logprint(f'  | Appended to Zarr {ZARR} successfully')
        
    except Exception as e:
        logprint(f'  X ERROR: {e}')
        raise e

logprint(f'  Appending data complete')

In [ ]:
dsz1 = xr.open_zarr(ZARR)
dsz1

In [ ]:
cubez = dsz1.surface_air_pressure.to_iris()
cubez

In [ ]:
deltas = cubez.coord('time').points[1:]-cubez.coord('time').points[0:-1]
deltas

In [ ]:
list(deltas).index(240)